In [1]:
import os
import sys
current_folder = os.path.dirname(os.path.abspath('__file__'))
parent_folder = os.path.dirname(current_folder)
grandparent_folder = os.path.dirname(parent_folder)
grandgrandparent_folder = os.path.dirname(grandparent_folder)
sys.path.append(grandgrandparent_folder)

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.MLP import MLP
from methods.methodsDataset.MLPDataset import MLPDataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1., 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_MLP = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [4]:
Dmin, Dmax = 0.1, 10
D_list = np.linspace(Dmin, Dmax, 1000)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())

U_sols = np.stack(U_sols)
D = torch.Tensor(D_list).view(-1, 1).to(device)
U = torch.Tensor(U_sols).to(device)

In [5]:
D_train, D_test, U_train, U_test = sklearn.model_selection.train_test_split(D, U,test_size=0.2, random_state=params_MLP['seed'])

In [6]:
def get_dataset():
    return D_train, D_test, U_train, U_test

In [7]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [8]:
def define_model(trial, input_size, output_size):
    n_layers = trial.suggest_int("n_layers", 1, 10)
    layers = [input_size]
    activations = []
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 52, 512)
        activation = trial.suggest_categorical("activation_l{}".format(i), ["tanh", "softplus", "relu"])
        layers += [out_features]
        activations.append(activation)
    layers += [output_size]
    params_MLP_trial = copy.deepcopy(params_MLP)
    params_MLP_trial['layer_dims'] = layers
    params_MLP_trial['activations'] = activations
    return MLP(params={'solver':None, 'method':params_MLP_trial})

In [9]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 1, 101)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    D_train, D_val, U_train, U_val = get_dataset()

    trainDataset = MLPDataset(x=D_train, y=U_train)
    valDataset = MLPDataset(x=D_val, y=U_val)

    batch_size = trial.suggest_int("batch_size", 32, 256)
    torch.manual_seed(params_MLP['seed'])
    trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
    valLoader = torch.utils.data.DataLoader(valDataset, batch_size=batch_size, shuffle=False)

    # Training of the model.
    loss_vals = []
    for epoch in range(1000):
        model.train()
        for i, data in enumerate(trainLoader):
            inputs, label = data
            optimizer.zero_grad()
            output = model(inputs)
            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()
            
        # Validation of the model.
        model.eval()
        loss_val = 0.
        with torch.no_grad():
            for i, data in enumerate(valLoader):
                inputs, label = data
                optimizer.zero_grad()
                output = model(inputs)
                loss_val += loss_fn(output, label).item()
        loss_val /= (i+1)
        loss_vals.append(loss_val)

        trial.report(min(loss_vals), epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return min(loss_vals)

In [10]:
sampler = TPESampler(seed=params_MLP['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-01 16:15:36,039] A new study created in memory with name: no-name-1d625898-a44d-4df8-8063-35c0bedbca63


In [ ]:
study.optimize(objective, n_trials=10, timeout=600)

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value

Use this to fill config_step_1.py

In [ ]:
print(activations, '\n', layer_dims, '\n', optimizer, '\n', lr)